In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('all-roberta-large-v1', device = 'cpu')
model = SentenceTransformer('all-MiniLM-L12-v2', device = 'cpu')



2025-04-04 10:43:07.822467: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 10:43:07.830655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743743587.839750  207345 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743743587.842506  207345 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743743587.849625  207345 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load the train data

In [3]:

# Load both CSVs
csv1 = pd.read_csv("rupay_good_queries.csv", header=None, names=["text"])
csv2 = pd.read_csv("upi_good_queries.csv", header=None, names=["text"])

relevant_queries = pd.concat([csv1, csv2])['text'].tolist()


In [4]:

relevant_embeddings = model.encode(relevant_queries, convert_to_tensor=True)

def is_relevant(query, threshold=0.6):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.cos_sim(query_embedding, relevant_embeddings)
    max_sim = similarities.max().item()
    return True if max_sim >= threshold else False



### Testing

In [5]:
def testing(filename):
	test_queries_df = pd.read_csv(filename, header=None, names=["text"])
	test_queries_df["Classification"] = test_queries_df["text"].apply(is_relevant)
	relevant_count = (test_queries_df["Classification"]).sum()
	not_relevant_count = len(test_queries_df) - relevant_count

	# Print results
	print(f"Relevant Queries: {relevant_count}")
	print(f"Not Relevant Queries: {not_relevant_count}")


In [6]:
testing("negative_queries.csv")
testing("simple.csv")

Relevant Queries: 0
Not Relevant Queries: 100
Relevant Queries: 131
Not Relevant Queries: 72
